### Simple example for performing symbolic regression for a set of points

In [5]:
from nesymres.architectures.model import Model
from nesymres.utils import load_metadata_hdf5
from nesymres.dclasses import FitParams, NNEquation, BFGSParams
from pathlib import Path
from functools import partial
import torch
from sympy import lambdify

In [6]:
## Set up BFGS load rom the hydra config yaml
bfgs = BFGSParams(
        activated= cfg.inference.bfgs.activated,
        n_restarts=cfg.inference.bfgs.n_restarts,
        add_coefficients_if_not_existing=cfg.inference.bfgs.add_coefficients_if_not_existing,
        normalization_o=cfg.inference.bfgs.normalization_o,
        idx_remove=cfg.inference.bfgs.idx_remove,
        normalization_type=cfg.inference.bfgs.normalization_type,
        stop_time=cfg.inference.bfgs.stop_time,
    )


In [7]:
## Use same set of configuration parameters of ours test set
import omegaconf
test_data = load_metadata_hdf5("../data/validation")
cfg = omegaconf.OmegaConf.load("../scripts/config.yaml")

params_fit = FitParams(word2id=test_data.word2id, 
                            id2word=test_data.id2word, 
                            una_ops=test_data.una_ops, 
                            bin_ops=test_data.bin_ops, 
                            total_variables=list(test_data.total_variables),  
                            total_coefficients=list(test_data.total_coefficients),
                            rewrite_functions=list(test_data.rewrite_functions),
                            bfgs=bfgs,
                            beam_size=cfg.inference.beam_size #This parameter is a tradeoff between accuracy and fitting time
                            )

In [8]:
weights_path = "../weights/100M.ckpt"

In [9]:
## Load architecture, set into eval mode, and pass the config parameters
model = Model.load_from_checkpoint(weights_path, cfg=cfg.architecture)
model.eval()
if torch.cuda.is_available(): 
  model.cuda()

In [10]:
fitfunc = partial(model.fitfunc,cfg_params=params_fit)

In [11]:
# Create points from an equation
number_of_points = 500
n_variables = 1

#To get best results make sure that your support inside the max and mix support
max_supp = cfg.dataset_train.fun_support["max"] 
min_supp = cfg.dataset_train.fun_support["min"]
X = torch.rand(number_of_points,len(list(test_data.total_variables)))*(max_supp-min_supp)+min_supp
X[:,n_variables:] = 0
target_eq = "x_1*sin(x_1)" #Use x_1,x_2 and x_3 as indepdent variables
X_dict = {x:X[:,idx].cpu() for idx, x in enumerate(test_data.total_variables)} #CHECK ME
y = lambdify(",".join(test_data.total_variables), target_eq)(**X_dict)

In [12]:
print("X shape: ", X.shape)
print("y shape: ", y.shape)

X shape:  torch.Size([500, 3])
y shape:  torch.Size([500])


In [13]:
output = fitfunc(X,y) 

/home/gem/repos/NeuralSymbolicRegressionThatScales/src/nesymres/architectures/model.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X,device=self.device).unsqueeze(0)
/home/gem/repos/NeuralSymbolicRegressionThatScales/src/nesymres/architectures/model.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y,device=self.device).unsqueeze(0)
Memory footprint of the encoder: 6.144e-05GB 

Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Loss constructed, starting new BFGS optmization...
Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values

In [14]:
output

{'all_bfgs_preds': ['((x_1)*(sin(x_1)))',
  '((x_1)*((cos(x_1))*(tan(x_1))))',
  '1.00000001222406*x_1*sin(x_1)'],
 'all_bfgs_loss': [0.0, 6.091805e-14, 0.0],
 'best_bfgs_preds': ['((x_1)*(sin(x_1)))'],
 'best_bfgs_loss': [0.0]}